In [44]:
###　必要な情報を登録するよーん

#APIキー
api_key = "自分の登録してください"

# 登録したい名前
name = "DaichiHorita"
# その人の特徴
userData = "nemusou"

In [34]:
### 人を登録する



import http.client, urllib.request, urllib.parse, urllib.error, base64
import json

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key,
}

params = json.dumps({
    "name":name,
    "userData":userData
})

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/persongroups/5000/persons", params, headers)
    response = conn.getresponse()
    data = json.loads(response.read().decode("utf-8"))
    print(data)
    personId = data["personId"]
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

{'personId': '15e736ca-961f-476c-9861-30f3e13b6bab'}


In [41]:
### 人の顔を登録するよーん



import http.client, urllib.request, urllib.parse, urllib.error, base64, json, struct

headers = {
    # Request headers
    'Content-Type': 'application/octet-stream',
    'Ocp-Apim-Subscription-Key': api_key,
}

# 登録したい画像を指定
png_data = open("test.png", "rb").read()
try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/persongroups/5000/persons/%s/persistedFaces" % personId, png_data, headers)
    response = conn.getresponse()
    data = json.loads(response.read().decode("utf-8"))
    print(data)
    persistedFaceId = data["persistedFaceId"]
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

{'persistedFaceId': '8dd8cd9c-937b-4038-bde3-985d79ee1f90'}
8dd8cd9c-937b-4038-bde3-985d79ee1f90


In [43]:
### 学習する


import http.client, urllib.request, urllib.parse, urllib.error, base64, json

headers = {
    'Ocp-Apim-Subscription-Key': api_key,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/persongroups/5000/train", params, headers)
    response = conn.getresponse()
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [53]:
### 学習データと照らし合わせるために、
### 比較画像から顔を抽出



import http.client, urllib.request, urllib.parse, urllib.error, base64, json

headers = {
    # Request headers
    'Content-Type': 'application/octet-stream',
    'Ocp-Apim-Subscription-Key': api_key,
}

# 比較したい画像
png_data_1 = open("images.jpeg", "rb").read()

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/detect" , png_data_1, headers)
    response = conn.getresponse()
    data = json.loads(response.read().decode("utf-8"))
    print(data)
    faceId = data[0]["faceId"]
    print(faceId)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

[{'faceId': 'd6c78fbc-b34d-4731-9150-232691c50c58', 'faceRectangle': {'left': 17, 'width': 143, 'top': 41, 'height': 143}}]
d6c78fbc-b34d-4731-9150-232691c50c58


In [66]:
### さっき顔を切り取ったやつを使って、学習データと比較



import http.client, urllib.request, urllib.parse, urllib.error, base64, json

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key,
}

params = json.dumps({
        "personGroupId":"5000" ,
        "faceIds": [faceId] ,
        "maxNumOfCandidatesReturned":1,
        "confidenceThreshold": 0.5
})

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/identify", params, headers)
    response = conn.getresponse()
    data = json.loads(response.read().decode("utf-8"))
    #print(data)
    result = data[0]["candidates"][0]["confidence"]
    result2 = result * 100
    print("この画像は%sパーセントの正答率です" % result2)
    
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

この画像は60.056パーセントの正答率です


In [67]:
### 学習データを確認する



import http.client, urllib.request, urllib.parse, urllib.error, base64, json

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': api_key,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("GET", "/face/v1.0/persongroups/5000/persons/%s" % personId, params, headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"personId":"15e736ca-961f-476c-9861-30f3e13b6bab","persistedFaceIds":["453f6635-a58e-4f6b-b785-4f5fde2d1b17","83d54327-6ca4-4773-8ba8-798735cbb82b","8dd8cd9c-937b-4038-bde3-985d79ee1f90","dfb72c55-5a64-42ff-97a3-222fdd83823c","e4ea035e-7411-4d0a-a7fd-1e7b5d56f514","eab31fb5-0441-446c-bed8-8fcc5a0c6da7","ef8162f9-042b-41f0-839a-a16adabbb978"],"name":"KoichiMatsumura","userData":"nemusou"}'
